# All TSA and loan clients with top-ups/cash-ins from Nov. 01, 2023 to date using debit cards

From: Mela Sombillo <csombillo1@tonikbank.com>
Sent: Wednesday, November 22, 2023 5:52 AM
To: Gaurav <gkumar@tonikbank.com>; Dwaipayan <dchakroborti@tonikbank.com>; Gurleen Singh <gsingh@tonikbank.com>
Cc: Fraud Operations DL <fraud-operations.dl@tonikbank.com>; Al Jason San Jose <asanjose@tonikbank.com>
Subject: Anti-Fraud Ops Request | 22NOV23

 

Hi @Gaurav and Team,

 

 

Good morning.

 

Can you please help us extract all TSA and loan clients with top-ups/cash-ins from Nov. 01, 2023 to date using debit cards?

 

We want to check and review these clients due to the recent issue discovered by Back Office Team related to our PayNamics channel. We need to make sure that there are no other fraudulent/suspicious accounts. Thank you.

 

 

Regards,

Mela

In [1]:
# Import Libraries

import pandas as pd 
import numpy as np 

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

In [8]:
# Query

sq = """
select cust_id
, tsaaccountnumber
, to_account_no
, txn_ts
, status
, trx_amount
, trx_fee_amount 
, total_trx_amount
, trx_type
, transfervia
, transfer_description
, error_details
, merchant_name
, channel
, subchannel
 from prj-prod-dataplatform.risk_credit_mis.channel_transactions
 where date(txn_ts) between '2023-11-01' and current_date()
 and coalesce(merchant_name, 'NA') like 'Debit Card'
 ;
 """

In [9]:
# Run the query to extract Data

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in DataFrame df is:\t{df.shape}")

Job ID a3c7eba5-09f3-423d-98fc-94e7ad68ecc2 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in DataFrame df is:	(2105, 15)


In [10]:
df.sample(5)

,cust_id,tsaaccountnumber,to_account_no,txn_ts,status,trx_amount,trx_fee_amount,total_trx_amount,trx_type,transfervia,transfer_description,error_details,merchant_name,channel,subchannel
1350,2292156,60822921560002,60822921560002,2023-11-03 09:06:44,Failed,500.00,12.5,512.50,CREDIT,Cashin - Paynamics,None,Webhook Updated,Debit Card,OTHER BANK DEBIT CARD,OTHER BANK DEBIT CARD
2070,2012378,60820123780002,60820123780002,2023-11-11 21:11:35,Failed,500.00,12.5,512.50,CREDIT,Cashin - Paynamics,None,Card Issuer Declined,Debit Card,OTHER BANK DEBIT CARD,OTHER BANK DEBIT CARD
647,2169117,60821691170002,60821691170002,2023-11-16 22:57:30,Failed,25060.34,0.0,25060.34,CREDIT,Cashin - Paynamics,None,"Transaction Failed, Exceeds Amount Limit",Debit Card,OTHER BANK DEBIT CARD,OTHER BANK DEBIT CARD
891,2047803,60820478030003,60820478030003,2023-11-17 17:58:39,Failed,2549.00,0.0,2549.00,CREDIT,Cashin - Paynamics,None,BIN restricted,Debit Card,OTHER BANK DEBIT CARD,OTHER BANK DEBIT CARD
1414,2291878,60822918780005,60822918780005,2023-11-03 09:45:38,Pending,20000.00,500.0,20500.00,CREDIT,Cashin - Paynamics,None,None,Debit Card,OTHER BANK DEBIT CARD,OTHER BANK DEBIT CARD


In [7]:
# Check the status counts
df.status.value_counts()

Pending    907
Failed     720
Success    478
Name: status, dtype: int64

In [20]:
# error_details 

df.error_details.value_counts()


Webhook Updated                                 435
Transaction Successful                          325
Query Successful                                123
BIN restricted                                   91
Card Issuer Declined                             87
Transaction Failed, Exceeds Amount Limit         47
Transaction Failed due to Insufficient funds     41
Transaction Successful with 3DS                  30
CVC is incorrect.                                 8
Transaction Status Update Failed                  5
Card Stolen                                       2
23                                                2
Restricted Card                                   1
Expired Card                                      1
Name: error_details, dtype: int64

In [ ]:
sum_of_value_counts = df['error_details'].value_counts().sum()
print(f'Sum of value counts: {sum_of_value_counts}')

In [13]:
# Check the max and min transaction date

print(f"The minimum transaction date is\t{df.txn_ts.min()}")
print(f"The maximum transaction date is\t{df.txn_ts.max()}")


The minimum transaction date is	2023-11-01 08:00:03
The maximum transaction date is	2023-11-18 22:46:45


In [14]:
# trx_type details

df.trx_type.value_counts()

CREDIT    2105
Name: trx_type, dtype: int64

In [15]:
# transfervia details

df.transfervia.value_counts()

Cashin - Paynamics    2105
Name: transfervia, dtype: int64

In [21]:
df.to_csv(r"AllTSAClientswithTopupcashinusingDebitcard.csv", index = False)